# Optimisation – Project 1

## Part 3.2
Median problem with weighted Euclidean distance

\begin{equation}
\min_{x \in \mathbb{R}} \sum_{i \in \mathcal{M}} v^i d_2(a^i, x)
\end{equation}

where $\mathcal{M} = \{1, \dots, m\}$ and $0 ≤ v^i \in \mathbb{R}, \; \forall i \in \mathcal{M}$.

We will here look into the Weiszfeld algorithm for solving the problem above.

## See overleaf for some of the solutions

## 4

In [78]:
import autograd.numpy as np
from autograd import grad
import time

## A solution for implementing an apropriate stopping criterion

In [166]:
def ourWeiszfeld(A, v, epsilon = 10**(-7)):
    """
    A = [a^1,..., a^i,..., a^m]^T   where   a^i = [a_1^i, a_2^i]
    Hence, A is a m times 2 matrix
    v = [v^1,..., v^i,..., v^m]^T   where   0<v^i \in \mathbb{R}, \forall i \in \mathcal{M} 
    epsilon > 0
    """
    m = len(A) # len(A) gives number of rows i.e. m
    
    # We define a test function to make the code leaner
    def testK(k):
        sum1 = 0
        sum2 = 0
        for i in range(m): 
            if i != k:
                dist = np.linalg.norm((A[:, k], A[:, i])) # d_2(*,*) euclidean distance
                sum1 += v[i]* (A[:, k].item(0) - A[:, i].item(0))/(dist)
                sum2 += v[i]* (A[:, k].item(1) - A[:, i].item(1))/(dist)
                
        result = np.sqrt(sum1**2 + sum2**2)
        return (result <= v[k])

    
    for k in range(m): 
        # theorem 1 test if the theorem is fulfilled for a "k"
        print(k)
        if testK(k): # do we need to pass the k? should rather not?
            x_star = A[:, k] # could just return A[k], but this is clearer
            return(x_star)
    
    # choose a starting point x = [x_1, x_2]^T, can be found solving the median problem with ||\cdot||_2^2
    x = np.array([0.5, 0.5])
    
    
    #############################
    # Functions for stopping ciretion
    def sigma(x):
        dist = np.linalg.norm(x[:, None] - A, axis = 0)
        return np.max(dist)
    
    def f_d2(x):
        return (np.sum(np.multiply(v,np.linalg.norm(x[:, None] - A, axis = 0))))

    gradient = grad(f_d2)
    
    def stoping_criterion(x):
        norm_grad = np.linalg.norm(gradient(x))
        sig = sigma(x)
        return ((norm_grad * sig)/(f_d2(x) - norm_grad * sig))
    
    while stoping_criterion(x) > epsilon:
        print(stoping_criterion(x))
        print(x)
        
        for j in range(len(x)):
            enum = 0
            deno = 0
            
            for i in range(m):
                dist = np.linalg.norm((np.squeeze(np.array(A[:, i])), x))
                enum += v[i]*A[i].item(j)/dist
                deno += v[i]/dist

            x[j] = enum/deno
        
    return x

In [183]:
#ourWeiszfeld(A, v)
def testK(k):
    sum1 = 0
    sum2 = 0
    for i in range(m):
        print(sum1)
        print(sum2)
        if i != k:
            dist = np.linalg.norm((A[:, k], A[:, i])) # d_2(*,*) euclidean distance
            sum1 += v[i] * (A[:, k].item(0) - A[:, i].item(0))/(dist)
            sum2 += v[i] * (A[:, k].item(1) - A[:, i].item(1))/(dist)

    result = np.sqrt(sum1**2 + sum2**2)
    print(result)
    return (result <= v[k])

m = len(A)
i = 8
A, A[:, i], A[:, i].item(0), v[i]

#testK(7)

(array([[0.1602515 , 0.23006771, 0.88370798, 0.778254  , 0.88261056,
         0.05853288, 0.28954831, 0.67149451, 0.42790053, 0.36432006],
        [0.7800156 , 0.78239727, 0.25341472, 0.29051009, 0.75126861,
         0.0195349 , 0.76071425, 0.83918901, 0.41237964, 0.6506515 ]]),
 array([0.42790053, 0.41237964]),
 0.4279005282024838,
 0.8406344699433242)

## Gradient descent algorithm

In [5]:
def steepestDescent(x_init, obj_func, max_iter = 10000, threshold = 10**(-10)):
    # We create the gradient function for the objective function using autograd
    gradient = grad(obj_func)
    
    # Here we initialize
    i = 0
    x = np.array(x_init) 
    rho = 0.4
    c = 0.6
    diff = np.full((len(x), 1), 100) # initialize some value
    while i < max_iter and diff.any() > threshold: # Can use same stopping criterion as above to measure time performance
        i += 1
        a = 1
        grad_k = gradient(x)
        # I make a pk for convenience 
        pk = -grad_k
        
        # The following is backtracking
        while obj_func(x + a*pk) > obj_func(x) + a*c*np.transpose(grad_k) @ pk: # i.e. repeat until not true
            a = rho*a
        
        # The new x is stored
        x = x + a*pk
        diff = abs(a*pk)
    return x

In [131]:
A = np.random.random(size = (2, 10))
A

array([[0.1602515 , 0.23006771, 0.88370798, 0.778254  , 0.88261056,
        0.05853288, 0.28954831, 0.67149451, 0.42790053, 0.36432006],
       [0.7800156 , 0.78239727, 0.25341472, 0.29051009, 0.75126861,
        0.0195349 , 0.76071425, 0.83918901, 0.41237964, 0.6506515 ]])

In [42]:
x = np.array([1., 2.])
v = np.random.rand(10)

In [7]:
x[:, None] - A
np.linalg.norm(x[:, None] - A, axis = 0)

array([1.2398241 , 1.81774632, 1.83631853, 1.69824264, 1.55523353,
       1.07466948, 1.86865503, 1.61629104, 1.67317693, 1.34577181])

In [10]:
def sigma(A, x):
    dist = np.linalg.norm(x[:, None] - A, axis = 0)
    return np.max(dist)

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d   # get a convex hull computation algorithm
 
 
 
#points = np.random.rand(10, 2)   # 30 random points in 2-D
#hull = ConvexHull(points)
 
#vecs = []
#for i in range(len(hull.vertices)):
 ##   element = points[hull.vertices[i]]
 #   element = element.tolist()
 #   vecs.append(element)
 
def distance(x_1,x_2):
    x_1 = np.array(x_1)
    x_2 = np.array(x_2)
    diff = x_1 - x_2
    return np.linalg.norm(diff, 2) 
 
 
def isCounterClockwise(n1,n2, n3):
    return ((n1 <n2 <n3) or (n3 < n1 < n2) or (n2 < n3 < n1))
 
def Area(vtxList, n1, n2, n3): # get area of triangle made of point 1,2, 3 from a list of vertices
    n = len(vtxList)
    vtx1 = vtxList[n1]
    vtx2 = vtxList[n2]
    vtx3 = vtxList[n3]
    # print(vtx1, vtx2, vtx3)
    a = distance(vtx1,vtx2)
    b = distance(vtx3,vtx2)
    c = distance(vtx1,vtx3)
    s = (a + b + c) / 2
    area = (s*(s-a)*(s-b)*(s-c)) ** 0.5
    if isCounterClockwise(n1,n2,n3):
        return round(area,3)
    else:
        return round(-area,3)
 
 
def GetAllAntiPodalPairs(vtxList):
    pairList = []
    n = len(vtxList)
    i_1 = 1
    i_n = 0
    i = i_n
    j = (i + 1)%n
    while (Area(vtxList,i, (i + 1)%n, (j + 1)%n) > Area(vtxList, i, (i + 1)%n, j)):
        j = (j + 1)%n
        j0 = j
    while (j != i_1):
        i = (i + 1)%n
        pairList.append([i, j])
        while ((Area(vtxList, i, (i + 1)%n, (j + 1)%n) > Area(vtxList, i, (i + 1)%n, j))):
            j = (j + 1)%n
            if (j != i_1):
                pairList.append([i, j])
        if (Area(vtxList, i, (i + 1)%n, (j + 1)%n) == Area(vtxList, i, (i + 1)%n, j)):
            if ((i, j) != (j0, i_n)):
                pairList.append([i, (j + 1)%n])
 
    return(pairList)
 
 
 
def findDiameter(polygon):
    maxx = 0
    antiPodalPairs = GetAllAntiPodalPairs(polygon)
    for pair in antiPodalPairs:
        pairPoint1 = polygon[pair[0]]
        pairPoint2 = polygon[pair[1]]
        candidate = distance(pairPoint1, pairPoint2)
        if (candidate > maxx):
            maxx = candidate
    return maxx
 
 
# EXAMPLE
 
polygonZ = [[0,0], [3,0], [2,1], [0,1]]
print(findDiameter(polygonZ))

3.1622776601683795


## 5

Compare using precision, speed and possibly stability
